In [1]:
import numpy as np
from sklearn import datasets
import warnings
warnings.filterwarnings('ignore')

In [2]:
def standard_scale(X):
    """Стандартизация одного признака"""
    mean = X.mean()
    std = X.std()
    return (X - mean) / std

def standard_scale2(X):
    """Стандартизация датасета целиком"""
    X_stand = X.copy()
    for i in range(X_stand.shape[1]):
        mean = X_stand[:, i].mean()
        std = X_stand[:, i].std()
        X_stand[:, i] = (X_stand[:, i] - mean) / std
    return X_stand

__1*.__ Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [3]:
def calc_logloss(y, y_pred):
    for i in range(len(y)):

      if y_pred[i]==1:
          y_pred[i] = 0.9999999999
      if y_pred[i]==0:
          y_pred[i] = 1e-10
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

__2.__ Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [4]:
# сгеренируем данные с помощью sklearn.datasets и стандартизируем признаки
X, y = datasets.make_classification(n_samples=100, n_features=4, n_informative=2,
                                    n_redundant=0, n_classes=2, random_state=1)
X_stand = standard_scale2(X)

In [5]:
def eval_model(X, y, iterations, eta=1e-4, log=True):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = 1 / (1 + np.exp(-z))
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0 and log==True:
            print(f'Iteration: {i}, Weights: {W}, Logloss: {err}')
    return W

In [6]:
eval_model(X_stand, y, iterations=1500, eta=0.3)

Iteration: 0, Weights: [ 0.49393621 -0.13412425  0.61792284  1.57066707], Logloss: 0.2875378351156201
Iteration: 150, Weights: [ 0.60864946  0.10858356 -0.1760802   3.67909521], Logloss: 0.10888624645403817
Iteration: 300, Weights: [ 0.78499261  0.22816412 -0.33383349  4.32275798], Logloss: 0.09771652505280905
Iteration: 450, Weights: [ 0.91860057  0.30630752 -0.44301962  4.71628326], Logloss: 0.09341541781024107
Iteration: 600, Weights: [ 1.02184007  0.36307047 -0.52633233  5.00224638], Logloss: 0.09111586046147434
Iteration: 750, Weights: [ 1.10410491  0.40717179 -0.59269246  5.22692153], Logloss: 0.08969411683357005
Iteration: 900, Weights: [ 1.17148433  0.44293381 -0.64720933  5.41120736], Logloss: 0.08873964713966036
Iteration: 1050, Weights: [ 1.22791845  0.4727629  -0.69305027  5.56650423], Logloss: 0.08806390153374599
Iteration: 1200, Weights: [ 1.27602393  0.49813779 -0.73228196  5.69982274], Logloss: 0.08756738162546121
Iteration: 1350, Weights: [ 1.31760004  0.52003899 -0.76

array([ 1.35370152,  0.53903496, -0.79597088,  5.91719996])

__3.__ Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [7]:
def calc_pred_proba(X, W):
    z = np.dot(X, W)
    y_pred_proba = 1 / (1 + np.exp(-z))
    return y_pred_proba

In [8]:
proba = calc_pred_proba(X_stand, eval_model(X_stand, y, iterations=1500, eta=0.3, log=False))
proba

array([4.53343519e-04, 1.72079657e-01, 3.78900800e-03, 5.23496208e-03,
       6.15141802e-04, 9.53423105e-01, 9.99440255e-01, 9.95933373e-01,
       9.95245005e-01, 9.01901370e-01, 9.95537838e-01, 3.81520984e-02,
       3.20239834e-03, 9.60303741e-01, 2.14236650e-03, 2.19786843e-03,
       9.99449773e-01, 1.08676425e-04, 5.68086709e-03, 9.99997044e-01,
       1.35568693e-02, 8.74513748e-04, 1.20878064e-02, 9.99913482e-01,
       9.73373546e-01, 9.97907954e-01, 6.96082706e-01, 9.99533867e-01,
       1.43175200e-01, 1.38275916e-02, 1.37161523e-04, 9.35938189e-01,
       9.99978045e-01, 5.23762801e-04, 1.32298666e-04, 8.59835437e-05,
       9.96202314e-01, 9.98501796e-01, 7.85030361e-03, 4.12839772e-04,
       4.25815298e-04, 9.99961918e-01, 9.99427697e-01, 9.22271728e-01,
       4.18584757e-03, 3.44224226e-03, 9.59306839e-01, 1.16321331e-03,
       9.94155336e-01, 3.79895046e-03, 8.26906402e-01, 9.96973668e-01,
       9.99288663e-01, 8.58897812e-01, 9.99785278e-01, 9.99756503e-01,
      

__4.__ Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [9]:
def calc_pred(X, W, threshold=0.5):
    y_pred_proba = calc_pred_proba(X, W)
    y_pred = np.where(y_pred_proba>threshold, 1, 0)
    return y_pred

In [10]:
y_pred = calc_pred(X_stand, eval_model(X_stand, y, iterations=1500, eta=0.3, log=False))
y_pred

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1])

__5*.__ Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [11]:
def confus_matrix(y_true, y_pred):
    TP = len([e for e in map(lambda x,y: (y==1) & (x == y), y_true, y_pred) if e])
    FP = len([e for e in map(lambda x,y: (y==1) & (x != y), y_true, y_pred) if e])
    TN = len([e for e in map(lambda x,y: (y==0) & (x == y), y_true, y_pred) if e])
    FN = len([e for e in map(lambda x,y: (y==0) & (x != y), y_true, y_pred) if e])
    matrix = np.array([[TP, FP], [FN, TN]])
    return matrix

def accuracy(y_true, y_pred):
    matrix = confus_matrix(y_true, y_pred)
    TP = matrix[0, 0]; FP = matrix[0, 1]; FN = matrix[1, 0]; TN = matrix[1, 1]
    return (TP + TN) / (TP + FP + TN + FN)

def precision(y_true, y_pred):
    matrix = confus_matrix(y_true, y_pred)
    TP = matrix[0, 0]; FP = matrix[0, 1]; FN = matrix[1, 0]; TN = matrix[1, 1]
    return TP / (TP + FP)

def recall(y_true, y_pred):
    matrix = confus_matrix(y_true, y_pred)
    TP = matrix[0, 0]; FP = matrix[0, 1]; FN = matrix[1, 0]; TN = matrix[1, 1]
    return TP / (TP + FN)

def fscore(y_true, y_pred, beta=1):
    matrix = confus_matrix(y_true, y_pred)
    TP = matrix[0, 0]; FP = matrix[0, 1]; FN = matrix[1, 0]; TN = matrix[1, 1]
    f = ((1 + beta**2) * TP) / (((1 + beta**2) * TP) + beta**2 * FN + FP)
    return  f

In [12]:
matrix_ = confus_matrix(y, y_pred)
accuracy_ = accuracy(y, y_pred)
precision_ = precision(y, y_pred)
recall_ = recall(y, y_pred)
f_score_ = fscore(y, y_pred)

print(f'Confusion matrix: \n{matrix_}', 
      f'Accuracy: {accuracy_:.4f}', 
      f'Precision: {precision_:.4f}', 
      f'Recall: {recall_:.4f}',
      f'F-score: {f_score_:.4f}', sep='\n'+'-'*20+'\n')

Confusion matrix: 
[[48  1]
 [ 1 50]]
--------------------
Accuracy: 0.9800
--------------------
Precision: 0.9796
--------------------
Recall: 0.9796
--------------------
F-score: 0.9796


__6.__ Могла ли модель переобучиться? Почему?  

Могла, если модель идеально подстроилась под все значения обучающего датасета